# 问题介绍

二维网格寻路算法在游戏中较为常见，它给定一个网格状拓扑图（包含可以移动的位置和障碍物）和图中的起点和终点坐标，寻找从起点到终点的最短路径。网络状拓扑图中的格点的邻居定义：可以是该格点上下左右四个方向的格点作为邻居，也可以加上左上右上左下右下四个相邻的格点作为邻居。本文中只考虑上下左右四个邻居的情况。  

In [ ]:
# 定义和查看输出的二维网格点。
import algviz

row, col = 10, 10
blocks = [
    (1, 2), (2, 3), (3, 3),
    (5, 4), (5, 5), (6, 5),
    (7, 6), (8, 6), (4, 5)
]
start = (3, 1)
end = (7, 8)

viz_example = algviz.Visualizer()
grid_example = viz_example.createTable(row, col, cell_size=20, name='Example')
for block in blocks:
    grid_example.mark(algviz.colors['silver'], block[0], block[1])
grid_example.mark(algviz.colors['green'], start[0], start[1])
grid_example.mark(algviz.colors['red'], end[0], end[1])
viz_example.display(0)

def markShortestPath(grid_, start_, end_):
    cur = end_
    while cur != start_:
        if cur != end_:
            grid_.mark(algviz.colors[0], cur[0], cur[1])
        if grid_[cur[0]][cur[1]] == '⬇':
            cur = (cur[0] + 1, cur[1])
        elif grid_[cur[0]][cur[1]] == '⬆':
            cur = (cur[0] - 1, cur[1])
        elif grid_[cur[0]][cur[1]] == '➡':
            cur = (cur[0], cur[1] + 1)
        elif grid_[cur[0]][cur[1]] == '⬅':
            cur = (cur[0], cur[1] - 1)
        else:
            break
    
def hamiltonDistance(pos1, pos2):
    return abs(pos1[0]-pos2[0]) + abs(pos1[1]-pos2[1])

# 广度优先搜索

和拓扑图的广度优先搜索类似，从起点开始每次向外缘扩展一个节点，直到遇到终点。在向外扩展时，为了避免重复访问相同的节点，需要辅助数组 `visit` 来记录节点是否被访问过，在扩展节点时，需要一个队列来记录当前待扩展节点。
为了记录路径，我们需要在搜索的过程中记录访问的每个点的前驱节点，体现在可视化效果上就是各个方向的箭头（PS:是时候支持一波emoji了），从起点指向终点。  
> 这里解释一下：因为深度优先搜索中同一层次的节点离起点的深度是相同的，所以在扩展中第一次遇到终点时就可以直接终止算法了（当然，如果要寻找所以的最短路径还需要继续扩展）。

**时间复杂度：** 最坏情况下需要探索图中的每个节点，所以时间复杂度为 $O(R \cdot C)$，其中 $R$ 网格节点行数， $C$ 为网格节点列数。  
**空间复杂度：** 需要辅助数组记录是否访问过节点，还需要队列记录待扩展节点，因此空间复杂度为 $O(R \cdot C)$。

In [ ]:
# 广度优先搜索算法实现。

def bFSFindPath(row, col, blocks, start, end):
    # 可视化初始化部分。
    viz = algviz.Visualizer(delay=2)
    grid = viz.createTable(row, col, cell_size=20)
    for block in blocks:
        grid.mark(algviz.colors['silver'], block[0], block[1])
    grid.mark(algviz.colors['green'], start[0], start[1])
    grid.mark(algviz.colors['red'], end[0], end[1])
    # 算法实现部分。
    visit = set()
    que = viz.createVector(show_index=False, name='Queue')
    que.append(start)
    while len(que) > 0:
        cur = que.pop(0)
        if cur == end:
            break
        if cur in visit:
            continue
        viz.display()
        grid[cur[0]][cur[1]]
        visit.add(cur)
        up = (cur[0]-1, cur[1])
        down = (cur[0]+1, cur[1])
        left = (cur[0], cur[1]-1)
        right = (cur[0], cur[1]+1)
        if up[0] >= 0 and up not in visit and up not in blocks:
            que.append(up)
            grid[up[0]][up[1]] = '⬇'
        if down[0] < row and down not in visit and down not in blocks:
            que.append(down)
            grid[down[0]][down[1]] = '⬆'
        if left[1] >=0 and left not in visit and left not in blocks:
            que.append(left)
            grid[left[0]][left[1]] = '➡'
        if right[1] < col and right not in visit and right not in blocks:
            que.append(right)
            grid[right[0]][right[1]] = '⬅'
    markShortestPath(grid, start, end)
    viz.display(0)
    viz.display(0)
    
bFSFindPath(row, col, blocks, start, end)

## 广度优先搜索总结

+ 从上面的演示中可以发现，广度优先搜索的效率是非常低的，会探索很多无用的节点。
+ ❓在搜索过程中，队列中有许多重复的节点，这影响吗？

# 贪心算法

贪心搜索算法每次向离终点最近的邻居点移动一个位置，直到到达终点，由于算法没有回溯的过程，因此跑起来非常快。但是有一个致命的缺陷，那就是对于某些障碍物的情况，贪心算法找到的**并不是最短路径**，局部最优不代表全局最优。

**时间复杂度：** 平均时间复杂度为 $O(R+C)$，但如果网格图像迷宫一样绕来绕去，时间复杂度可能上升至 $O(R \cdot C)$。  
**空间复杂度：** 不需要额外的储存空间来记录节点等，因此空间复杂度为 $O(1)$。

In [ ]:
# 贪心算法实现代码。

def greedyFindPath(row, col, blocks, start, end):
    # 可视化初始化部分。
    viz = algviz.Visualizer(delay=1)
    grid = viz.createTable(row, col, cell_size=20)
    for block in blocks:
        grid.mark(algviz.colors['silver'], block[0], block[1])
    grid.mark(algviz.colors['green'], start[0], start[1])
    grid.mark(algviz.colors['red'], end[0], end[1])
    # 算法实现部分。
    cur = start
    visit = set()
    while cur != end:
        viz.display()
        grid[cur[0]][cur[1]]
        visit.add(cur)
        min_dist = row + col
        up = (cur[0]-1, cur[1])
        down = (cur[0]+1, cur[1])
        left = (cur[0], cur[1]-1)
        right = (cur[0], cur[1]+1)
        if up[0] >= 0 and up not in visit and up not in blocks:
            dist = hamiltonDistance(up, end)
            if dist < min_dist:
                min_dist = dist
                cur = up
            grid[up[0]][up[1]] = '⬇'
        if down[0] < row and down not in visit and down not in blocks:
            dist = hamiltonDistance(down, end)
            if dist < min_dist:
                min_dist = dist
                cur = down
            grid[down[0]][down[1]] = '⬆'
        if left[1] >=0 and left not in visit and left not in blocks:
            dist = hamiltonDistance(left, end)
            if dist < min_dist:
                min_dist = dist
                cur = left
            grid[left[0]][left[1]] = '➡'
        if right[1] < col and right not in visit and right not in blocks:
            dist = hamiltonDistance(right, end)
            if dist < min_dist:
                min_dist = dist
                cur = right
            grid[right[0]][right[1]] = '⬅'
    markShortestPath(grid, start, end)
    viz.display(0)
    viz.display(0)

greedyFindPath(row, col, blocks, start, end)

## 贪心算法总结

+ 观察可以发现，这里通过贪心算法找到的路径并不是最短路径，说明贪心的思路是错误的。

# A\*算法

A\*算法结合了广度优先搜索算法和贪心算法的优点，它以广度优先搜索算法为基础框架，但是在扩展节点时不再按照固定的顺序（如上下左右）来进行，而是考虑到了邻居节点与终点之间的距离。这样既可以使算法能够优先向终点方向探索，又不至于因为障碍物的阻挡而得到错误的路径。为了记录每个待扩展节点及其到终点的距离，需要使用**优先队列**的数据结构来保存待扩展节点，而每次拓展时，都从优先队列中取出离终点最近的节点。  
> 注：在算法实现中，为了更加直观的可视化效果，我们使用 `Vector` 来代替优先队列，通过将新的节点插入到合适的位置来实现相同的效果。

**时间复杂度：** 和广度搜索算法类似，最快情况下的时间复杂度为 $O(R \cdot C)$，但平均情况下的时间复杂度要比广度优先搜索好。   
**空间复杂度：** 需要记录节点是否已被访问过，还有优先队列也占用空间，因此空间复杂度为 $O(R \cdot C)$。

In [ ]:
# A*算法实现代码。

# 该函数按照优先级将元素插入队列的相应位置。
def insertQueue(que_, que_keys_, k, v):
    for i in range(len(que_keys_)):
        if k < que_keys_[i]:
            que_.insert(i, v)
            que_keys_.insert(i, k)
            return
    que_.append(v)
    que_keys_.append(k)

def aStarFindPath(row, col, blocks, start, end):
    # 可视化初始化部分。
    viz = algviz.Visualizer(2)
    grid = viz.createTable(row, col, cell_size=20)
    for block in blocks:
        grid.mark(algviz.colors['silver'], block[0], block[1])
    grid.mark(algviz.colors['green'], start[0], start[1])
    grid.mark(algviz.colors['red'], end[0], end[1])
    # 算法实现部分。
    visit = set()
    que = viz.createVector(data=[start], name='PriorityQueue', show_index=False)
    que_keys = [0]    # 用于辅助对que中的元素进行排序。
    while len(que) > 0:
        cur = que.pop(0);que_keys.pop(0)
        if cur == end:
            break
        if cur in visit:
            continue
        viz.display()
        grid[cur[0]][cur[1]]
        visit.add(cur)
        up = (cur[0]-1, cur[1])
        down = (cur[0]+1, cur[1])
        left = (cur[0], cur[1]-1)
        right = (cur[0], cur[1]+1)
        cur_dist = hamiltonDistance(cur, start)
        if up[0] >= 0 and up not in visit and up not in blocks:
            dist = cur_dist + hamiltonDistance(up, end)
            insertQueue(que, que_keys, dist, up)
            grid[up[0]][up[1]] = '⬇'
        if down[0] < row and down not in visit and down not in blocks:
            dist = cur_dist + hamiltonDistance(down, end)
            insertQueue(que, que_keys, dist, down)
            grid[down[0]][down[1]] = '⬆'
        if left[1] >=0 and left not in visit and left not in blocks:
            dist = cur_dist + hamiltonDistance(left, end)
            insertQueue(que, que_keys, dist, left)
            grid[left[0]][left[1]] = '➡'
        if right[1] < col and right not in visit and right not in blocks:
            dist = cur_dist + hamiltonDistance(right, end)
            insertQueue(que, que_keys, dist, right)
            grid[right[0]][right[1]] = '⬅'
    markShortestPath(grid, start, end)
    viz.display(0)
    viz.display(0)

aStarFindPath(row, col, blocks, start, end)

# 参考链接

+ https://www.redblobgames.com/pathfinding/a-star/introduction.html